# Imports 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.patches import Ellipse
import seaborn as sns
from math import *
import matplotlib.pylab as pyl
import pickle
import swifter
import warnings
import plotly.express as px
from itertools import chain
import scipy.stats as sps
from tqdm import tqdm
from unidecode import unidecode
import re
from pathlib import Path
from tqdm.notebook import tqdm
import pandas as pd
import itertools


In [3]:
import warnings
warnings.filterwarnings('ignore', category=pd.io.pytables.PerformanceWarning)

In [4]:
#pd.set_option('max_colwidth', 999)
pd.set_option('display.max_columns', 1000)
pd.set_option("display.max_rows", 3000)

# Data 

In [5]:
# Loading clusters with VAEP values as a list of dataframes
cluster_names = [
    'rll', 'rlll', 'rlr', 'rrl', 'rrll', 'rrlll', 'rrlr', 'rrr', 'rrrl',
    'rrrll', 'rrrlr', 'rrrr', 'rrrrl'
]
df_clusters = list()
for name in cluster_names:
    df_clusters.append(
        pd.read_pickle(
            f'../data/clusters/clusters_vaep/cluster_{name}.pkl'
        ))

In [6]:
# Reseting the indexes of dfs in df_clusters
for df in df_clusters:
    df.reset_index(drop=True, inplace = True)

# Creating Player-Metrics Dataset

In [7]:
def player_metrics_func(l):
    player_metrics = pd.DataFrame(columns=[
        'player_name',
        'team',
        'backline_footedness',
        'backline',
        'position',
        'passes',
        'accpass',
        'accpassloc',
        'inaccpassloc',
        'accpassvaep',
        'inaccpassvaep',
        'accpassoff',
        'inaccpassoff',
        'accpassdef',
        'inaccpassdef',
        ])
    pos = [
        'RB',
        'R_CB',
        'L_CB',
        'LB',
        'RWB',
        'RCB',
        'CB',
        'LCB',
        'LWB',
        ]
    for df in l:
        for i in tqdm(range(len(df))):
            if df['backline'][i] == 4:
                s1 = list()
                for p in pos[0:4]:
                    p1 = p + '_'
                    s = [[df[p][i]], [df['team'][i]], [df['footedness'
                         ][i]], [df['backline'][i]], [p], df.loc[i,
                         list(df.columns[df.columns.str.startswith(p1)])].values]
                    s = list(itertools.chain(*s))
                    s1.append(s)
                s1 = np.vstack(s1)
                player_metrics = pd.concat([player_metrics,
                        pd.DataFrame(s1,
                        columns=player_metrics.columns)], axis=0)
            elif df['backline'][i] == 3:

                s1 = list()
                for p in pos[5:8]:
                    p1 = p + '_'
                    s = [[df[p][i]], [df['team'][i]], [df['footedness'
                         ][i]], [df['backline'][i]], [p], df.loc[i,
                         list(df.columns[df.columns.str.startswith(p1)])].values]
                    s = list(itertools.chain(*s))
                    s1.append(s)
                s1 = np.vstack(s1)
                player_metrics = pd.concat([player_metrics,
                        pd.DataFrame(s1,
                        columns=player_metrics.columns)], axis=0)
            elif df['backline'][i] == 5:

                s1 = list()
                for p in pos[4:]:
                    p1 = p + '_'
                    s = [[df[p][i]], [df['team'][i]], [df['footedness'
                         ][i]], [df['backline'][i]], [p], df.loc[i,
                         list(df.columns[df.columns.str.startswith(p1)])].values]
                    s = list(itertools.chain(*s))
                    s1.append(s)
                s1 = np.vstack(s1)
                player_metrics = pd.concat([player_metrics,
                        pd.DataFrame(s1,
                        columns=player_metrics.columns)], axis=0)

    return player_metrics


In [8]:
player_metrics = player_metrics_func(df_clusters)
print(player_metrics.shape)
player_metrics.reset_index(drop=True, inplace=True)


(2885, 15)


In [9]:
player_metrics.tail(20)

,player_name,team,backline_footedness,backline,position,passes,accpass,accpassloc,inaccpassloc,accpassvaep,inaccpassvaep,accpassoff,inaccpassoff,accpassdef,inaccpassdef
2865,KyleNaughton,Swansea,right-right-right-right-left,5,RWB,39,27,"[[[30.16, 6.12], [14.56, 8.84]], [[34.32, 2.04...","[[[32.24, 4.76], [35.36, 6.12]], [[33.28, 6.8]...","[-0.0016814813716337085, 0.0019396115094423294...","[-0.005136714782565832, -0.004266716074198484,...","[-0.0001473438460379839, 0.0022924211807549, -...","[-0.001564960228279233, -0.0015721209347248077...","[-0.0015341375255957246, -0.000352809671312570...","[-0.003571754554286599, -0.0026945951394736767..."
2866,MikevanderHoorn,Swansea,right-right-right-right-left,5,RCB,27,18,"[[[30.16, 21.76], [40.56, 14.96]], [[30.16, 16...","[[[60.32, 5.44], [70.72, 21.76]], [[21.84, 12....","[0.0004018775653094053, 0.012618033215403557, ...","[-0.0022834830451756716, 0.0031238358933478594...","[-0.0002832259051501751, 0.00231059268116951, ...","[-0.0007665012963116169, -0.000516334781423211...","[0.0006851034704595804, 0.010307440534234047, ...","[-0.0015169817488640547, 0.0036401706747710705..."
2867,FedericoFernndez,Swansea,right-right-right-right-left,5,CB,17,14,"[[[31.2, 33.32], [30.16, 21.76]], [[41.6, 16.3...","[[[52.0, 32.64], [71.76, 38.76]], [[27.04, 33....","[0.0016384110786020756, 0.0042571136727929115,...","[-0.003345789387822151, 0.00464475154876709, 0...","[0.0018189093098044395, 0.004284799564629793, ...","[-0.0008325581438839436, 0.0011699552414938807...","[-0.00018049823120236397, -2.7685891836881638e...","[-0.0025132312439382076, 0.0034747961908578873..."
2868,AlfieMawson,Swansea,right-right-right-right-left,5,LCB,21,19,"[[[32.24, 41.48], [31.2, 33.32]], [[26.0, 34.0...","[[[36.4, 49.64], [83.2, 68.0]], [[62.4, 54.4],...","[0.002098622266203165, 2.10154103115201e-05, 0...","[-0.0035038923379033804, -0.0006132156122475863]","[0.00207380554638803, 4.5678578317165375e-05, ...","[-0.0029103411361575127, -0.0009922496974468231]","[2.4816719815135002e-05, -2.4663168005645275e-...","[-0.0005935512017458677, 0.00037903408519923687]"
2869,MartinOlsson,Swansea,right-right-right-right-left,5,LWB,24,13,"[[[82.16, 63.92], [95.68, 62.56]], [[64.48, 62...","[[[35.36, 59.16], [82.16, 59.16]], [[46.8, 63....","[0.004542926326394081, 0.006097947712987661, 0...","[-0.0038077887147665024, -0.004220862872898579...","[0.004744295030832291, 0.0096541503444314, 0.0...","[-0.0025481916964054108, -0.003968306817114353...","[-0.0002013689372688532, -0.003556202631443739...","[-0.0012595969019457698, -0.000252555822953581..."
2870,EzequielSchelotto,Brighton,right-right-right-right-left,5,RWB,39,32,"[[[15.6, 6.12], [11.44, 18.36]], [[66.56, 17.6...","[[[101.92, 16.32], [95.68, 53.04]], [[92.56, 6...","[-6.912159733474255e-05, 0.0080186165869236, 0...","[-0.03959555923938751, -0.020668337121605873, ...","[-0.0001241816207766533, 0.00941790547221899, ...","[-0.035406287759542465, -0.0330783911049366, -...","[5.5060023441910744e-05, -0.001399289350956678...","[-0.004189271479845047, 0.012410053983330727, ..."
2871,ConnorGoldson,Brighton,right-right-right-right-left,5,RCB,37,29,"[[[20.8, 13.6], [15.6, 34.0]], [[31.2, 20.4], ...","[[[11.44, 18.36], [46.8, 17.68]], [[32.24, 19....","[-0.0007745048496872187, 0.0020564801525324583...","[-0.014456800185143948, -0.01597249135375023, ...","[-0.0011167689226567745, 0.0009175087325274944...","[-0.0008814043831080198, -0.012202132493257523...","[0.00034226407296955585, 0.0011389714200049639...","[-0.013575395569205284, -0.003770359791815281,..."
2872,ShaneDuffy,Brighton,right-right-right-right-left,5,CB,31,28,"[[[21.84, 47.6], [30.16, 36.72]], [[20.8, 33.3...","[[[20.8, 59.84], [28.08, 54.4]], [[26.0, 27.88...","[-0.00019680149853229523, 0.012086118571460247...","[-0.012083248235285282, -0.012766489759087563,...","[0.0024775743950158358, -0.0046207597479224205...","[0.0016831893008202314, -0.000304842134937644,...","[-0.002674375893548131, 0.016706878319382668, ...","[-0.013766

In [10]:
def extract_foot(a,b):
    pos1 = ['RB', 'R_CB', 'L_CB', 'LB']
    pos2 = ['RWB', 'RCB', 'CB', 'LCB', 'LWB']
    a = a.split('-')
    if(len(a)==4):
        return a[pos1.index(b)]
    elif(len(a)==5):
        return a[pos2.index(b)]
    elif(len(a)==3):
        return a[pos2.index(b) - 1]

In [11]:
player_metrics['footedness'] = player_metrics.apply(lambda x: extract_foot(x.backline_footedness, x.position), axis=1)
player_metrics.drop(columns = ['backline_footedness'], inplace=True)

In [12]:
player_metrics.head(5)

,player_name,team,backline,position,passes,accpass,accpassloc,inaccpassloc,accpassvaep,inaccpassvaep,accpassoff,inaccpassoff,accpassdef,inaccpassdef,footedness
0,SteveCook,Bournemouth,3,RCB,40,27,"[[[64.48, 19.72], [64.48, 4.76]], [[46.8, 11.5...","[[[27.04, 4.76], [39.52, 4.76]], [[34.32, 8.16...","[-0.0004391773836687207, 0.002637155819684267,...","[-0.003568546148017049, 0.003484044224023819, ...","[-0.000647583045065403, 0.0028843628242611885,...","[-0.0006589896511286497, 0.0006538690067827702...","[0.00020840566139668226, -0.000247207004576921...","[-0.002909556496888399, 0.002830175217241049, ...",right
1,NathanAke,Bournemouth,3,CB,30,24,"[[[48.88, 36.72], [39.52, 54.4]], [[6.24, 3.4]...","[[[28.08, 22.44], [55.12, 22.44]], [[31.2, 25....","[-0.0029666530899703503, 0.008709289133548737,...","[-0.0024040783755481243, 0.004382790066301823,...","[-0.0034658201038837433, 0.0011803375091403723...","[-0.0005394387990236282, 0.0018933985847979784...","[0.0004991668974980712, 0.007528951391577721, ...","[-0.001864639576524496, 0.002489391714334488, ...",left
2,TyroneMings,Bournemouth,3,LCB,45,38,"[[[32.24, 47.6], [68.64, 48.96]], [[39.52, 54....","[[[33.28, 56.44], [65.52, 48.28]], [[19.76, 36...","[0.006780868396162987, -0.0012391717173159122,...","[-0.001953032799065113, -0.02682320401072502, ...","[0.0070403809659183025, -0.0008301073685288429...","[-0.0014317366294562817, -0.002288793213665485...","[-0.0002595123369246721, -0.000409064348787069...","[-0.0005212961696088314, -0.02453441172838211,...",left
3,DeclanRice,West Ham,3,RCB,35,26,"[[[41.6, 18.36], [43.68, 33.32]], [[48.88, 14....","[[[28.08, 12.92], [68.64, 28.56]], [[36.4, 5.4...","[0.000787452154327184, 0.006387726403772831, 0...","[-0.0018625393277034163, -0.005176905076950788...","[0.0003065122291445732, 0.006481693126261234, ...","[-0.000464398181065917, -0.00370267266407609, ...","[0.00048093992518261075, -9.396695531904697e-0...","[-0.0013981411466374993, -0.001474232412874698...",right
4,AngeloOgbonna,West Ham,3,CB,25,24,"[[[24.96, 31.96], [40.56, 17.0]], [[38.48, 44....","[[[33.28, 35.36], [75.92, 46.24]]]","[0.005705440416932106, -7.556541822850704e-05,...",[-0.004009038209915161],"[0.001944628544151783, -9.733764454722404e-05,...",[-0.0013899148907512426],"[0.003760811872780323, 2.1772226318717003e-05,...",[-0.002619123086333275],left


In [14]:
df = player_metrics.groupby(['player_name', 'team', 'position', 'footedness'])['passes', 'accpass',
       'accpassloc', 'inaccpassloc', 'accpassvaep', 'inaccpassvaep', 'accpassoff',
       'inaccpassoff', 'accpassdef', 'inaccpassdef'].sum().reset_index()
df

,player_name,team,position,footedness,passes,accpass,accpassloc,inaccpassloc,accpassvaep,inaccpassvaep,accpassoff,inaccpassoff,accpassdef,inaccpassdef
0,AaronCresswell,West Ham,LB,left,275,203,"[[[11.44, 47.6], [10.4, 48.96]], [[21.84, 58.4...","[[[22.88, 65.28], [31.2, 63.24]], [[24.96, 61....","[-0.009422648698091507, 0.01328663807362318, 0...","[-0.003516178810968995, 0.0020406385883688927,...","[-0.002352543640881777, 0.0017920035170391202,...","[-0.001053002430126071, 0.0013813287951052189,...","[-0.00707010505720973, 0.011494634672999382, -...","[-0.002463176380842924, 0.0006593097932636738,..."
1,AaronCresswell,West Ham,LCB,left,796,644,"[[[27.04, 51.68], [55.12, 64.6]], [[65.52, 61....","[[[62.4, 51.0], [78.0, 49.64]], [[79.04, 61.2]...","[-0.0012921183370053768, 0.003276045434176922,...","[-0.007468235678970814, -0.010166996158659458,...","[0.0015182411298155785, 0.003629088867455721, ...","[-0.005851619876921177, -0.008341009728610516,...","[-0.0028103594668209553, -0.000353043549694120...","[-0.001616615685634315, -0.001825986779294908,..."
2,AaronWanBissaka,Crystal Palace,RB,right,226,165,"[[[60.32, 11.56], [70.72, 8.84]], [[55.12, 11....","[[[86.32, 22.44], [87.36, 25.84]], [[98.8, 10....","[0.004373389296233654, -0.002335744444280863, ...","[-0.024122614413499832, -0.01690908707678318, ...","[0.004053536802530289, -0.0032445229589939117,...","[-0.021383047103881836, -0.016634516417980194,...","[0.0003198523772880435, 0.0009087783982977271,...","[-0.002739568240940571, -0.0002745708916336298..."
3,AdamSmith,Bournemouth,LB,right,146,120,"[[[15.6, 59.84], [22.88, 54.4]], [[19.76, 61.2...","[[[67.6, 13.6], [74.88, 33.32]], [[44.72, 2.72...","[-0.003807531204074621, 0.0007853175047785044,...","[-0.007521173916757107, 0.0010863570496439934,...","[-0.0014487234875559807, 0.0004428883548825979...","[-0.004603862296789885, 0.0001980820670723915,...","[-0.0023588077165186405, 0.0003424291498959064...","[-0.002917311619967222, 0.0008882749825716019,..."
4,AdamSmith,Bournemouth,RB,right,451,387,"[[[28.08, 4.76], [40.56, 20.4]], [[31.2, 13.6]...","[[[55.12, 4.08], [62.4, 6.8]], [[35.36, 4.08],...","[0.006689831148833036, -0.0021313228644430637,...","[-0.01221891026943922, 0.00034937215968966484,...","[0.003623384051024914, -0.0014893298503011465,...","[-0.011155590415000916, 0.001172496471554041, ...","[0.0030664470978081226, -0.0006419930141419172...","[-0.0010633200872689486, -0.000823124311864376..."
5,AdamSmith,Bournemouth,RWB,right,17,10,"[[[98.8, 19.72], [93.6, 25.84]], [[33.28, 3.4]...","[[[57.2, 2.04], [53.04, 2.72]], [[28.08, 1.36]...","[0.034581780433654785, -0.0024608569219708443,...","[-0.023536473512649536, -0.008208570070564747,...","[0.03300246596336365, -0.000673642847687006, 0...","[-0.023219434544444084, -0.003743352135643363,...","[0.0015793144702911377, -0.00178721419069916, ...","[-0.00031703885179013014, -0.00446521816775202..."
6,AdrianMariappa,Watford,LB,right,40,33,"[[[61.36, 65.28], [68.64, 60.52]], [[65.52, 64...","[[[36.4, 59.84], [69.68, 53.04]], [[34.32, 59....","[0.005218063481152058, 0.004832935053855181, 0...","[0.0016785343177616596, 0.00020781706552952528...","[0.005568218417465687, 0.004288358613848686, 0...","[0.0016385330818593502, -0.0002384964609518647...","[-0.000350155052728951, 0.0005445763235911727,...","[4.000123590230942e-05, 0.00044631352648139, 0..."
7,AdrianMariappa,Watford,L_CB,right,179,144,"[[[36.4, 45.56], [74.88, 48.28]], [[30.16, 46....","[[[43.68, 20.4], [42.64, 26.52]], [[27.04, 59....","[0.01236659474670887, 0.00999591127038002, -0....","[-0.018949441611766815, 0.00031638285145163536...","[0.007915441878139973, 0.009581996127963066, 0...","[0.0008363674860447645, -0.0007496089674532413...","[0.0044511533342301846, 0.0004139149095863104,...","[-0.01978580839931965, 0.0010659918189048767, ..."
8,AdrianMariappa,Watford,RB,right,92,67,"[[[65.52, 2.04], [73.84, 2.04]], [[61.36, 4.08...","[[[45.76, 4.08], [62.4, 3.4]], [[60.32, 2.04],...","[-0.010988935828208923, -0.00434715

In [15]:
df.to_pickle('../data/defender_clustering_dataset.pkl')